In [1]:
from numpy import array
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
macau = pd.read_csv('Datasets/Macau-2022.csv', index_col='date', parse_dates = ['date'])

In [4]:
macau.head()

,arrival
date,
2011-01-01,2076066
2011-02-01,2164249
2011-03-01,2190865
2011-04-01,2338449
2011-05-01,2295551


In [5]:
mc_values = macau.values

In [6]:
n_steps_in = 4
n_steps_out = 2

In [7]:
X, y = split_sequence(mc_values, n_steps_in, n_steps_out)

In [8]:
n_features = 1

In [9]:
X = X.reshape(X.shape[0],X.shape[1], n_features)

In [10]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [11]:
model.fit(X, y, epochs=50, verbose=2)

Epoch 1/50
5/5 - 2s - loss: 5734391087104.0000 - 2s/epoch - 413ms/step
Epoch 2/50
5/5 - 0s - loss: 5503296471040.0000 - 23ms/epoch - 5ms/step
Epoch 3/50
5/5 - 0s - loss: 5312642809856.0000 - 21ms/epoch - 4ms/step
Epoch 4/50
5/5 - 0s - loss: 5077397405696.0000 - 22ms/epoch - 4ms/step
Epoch 5/50
5/5 - 0s - loss: 4862955749376.0000 - 22ms/epoch - 4ms/step
Epoch 6/50
5/5 - 0s - loss: 4640164806656.0000 - 21ms/epoch - 4ms/step
Epoch 7/50
5/5 - 0s - loss: 4196472979456.0000 - 21ms/epoch - 4ms/step
Epoch 8/50
5/5 - 0s - loss: 3842235170816.0000 - 22ms/epoch - 4ms/step
Epoch 9/50
5/5 - 0s - loss: 3623957299200.0000 - 21ms/epoch - 4ms/step
Epoch 10/50
5/5 - 0s - loss: 2396950364160.0000 - 21ms/epoch - 4ms/step
Epoch 11/50
5/5 - 0s - loss: 1634336899072.0000 - 20ms/epoch - 4ms/step
Epoch 12/50
5/5 - 0s - loss: 933189451776.0000 - 20ms/epoch - 4ms/step
Epoch 13/50
5/5 - 0s - loss: 443272331264.0000 - 21ms/epoch - 4ms/step
Epoch 14/50
5/5 - 0s - loss: 382493458432.0000 - 20ms/epoch - 4ms/step
Epoc

In [12]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [13]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

619120
587575
 0.21 


In [14]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [15]:
model.fit(X, y, epochs=50, verbose=2)

Epoch 1/50
5/5 - 2s - loss: 5754889175040.0000 - 2s/epoch - 390ms/step
Epoch 2/50
5/5 - 0s - loss: 5243285798912.0000 - 21ms/epoch - 4ms/step
Epoch 3/50
5/5 - 0s - loss: 4683179491328.0000 - 21ms/epoch - 4ms/step
Epoch 4/50
5/5 - 0s - loss: 3666025119744.0000 - 24ms/epoch - 5ms/step
Epoch 5/50
5/5 - 0s - loss: 2851964977152.0000 - 28ms/epoch - 6ms/step
Epoch 6/50
5/5 - 0s - loss: 2275833806848.0000 - 31ms/epoch - 6ms/step
Epoch 7/50
5/5 - 0s - loss: 1671797407744.0000 - 23ms/epoch - 5ms/step
Epoch 8/50
5/5 - 0s - loss: 1087687557120.0000 - 23ms/epoch - 5ms/step
Epoch 9/50
5/5 - 0s - loss: 591539535872.0000 - 23ms/epoch - 5ms/step
Epoch 10/50
5/5 - 0s - loss: 339180716032.0000 - 25ms/epoch - 5ms/step
Epoch 11/50
5/5 - 0s - loss: 241609687040.0000 - 25ms/epoch - 5ms/step
Epoch 12/50
5/5 - 0s - loss: 221425205248.0000 - 27ms/epoch - 5ms/step
Epoch 13/50
5/5 - 0s - loss: 215000367104.0000 - 23ms/epoch - 5ms/step
Epoch 14/50
5/5 - 0s - loss: 190644830208.0000 - 21ms/epoch - 4ms/step
Epoch 1

In [16]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [17]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

832348
724031
 0.30 


In [18]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [19]:
model.fit(X, y, epochs=50, verbose=0)

In [20]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [21]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

301149
549645
 0.73 


In [22]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [23]:
y = y.reshape((y.shape[0], y.shape[1], n_features))

In [24]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

In [25]:
model.fit(X, y, epochs=50, verbose=2)

Epoch 1/50
5/5 - 2s - loss: 5760333381632.0000 - 2s/epoch - 452ms/step
Epoch 2/50
5/5 - 0s - loss: 5406897733632.0000 - 19ms/epoch - 4ms/step
Epoch 3/50
5/5 - 0s - loss: 5126699352064.0000 - 19ms/epoch - 4ms/step
Epoch 4/50
5/5 - 0s - loss: 4807661715456.0000 - 21ms/epoch - 4ms/step
Epoch 5/50
5/5 - 0s - loss: 4383593725952.0000 - 21ms/epoch - 4ms/step
Epoch 6/50
5/5 - 0s - loss: 3868099084288.0000 - 18ms/epoch - 4ms/step
Epoch 7/50
5/5 - 0s - loss: 3266831974400.0000 - 20ms/epoch - 4ms/step
Epoch 8/50
5/5 - 0s - loss: 2462545084416.0000 - 19ms/epoch - 4ms/step
Epoch 9/50
5/5 - 0s - loss: 1632008536064.0000 - 19ms/epoch - 4ms/step
Epoch 10/50
5/5 - 0s - loss: 1041910857728.0000 - 25ms/epoch - 5ms/step
Epoch 11/50
5/5 - 0s - loss: 815981723648.0000 - 27ms/epoch - 5ms/step
Epoch 12/50
5/5 - 0s - loss: 847725264896.0000 - 29ms/epoch - 6ms/step
Epoch 13/50
5/5 - 0s - loss: 776248950784.0000 - 29ms/epoch - 6ms/step
Epoch 14/50
5/5 - 0s - loss: 617616441344.0000 - 28ms/epoch - 6ms/step
Epoch

In [26]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [27]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

591704
576177
 0.27 
